In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



# 1) Prepare counts
agent_counts = (
    df
    .groupby(['Antagonist_type_general','system_type'], observed=True)
    .size()
    .unstack(fill_value=0)
    .loc[['Virus','Eukaryotic parasite','Pest','Bacteria']]
)

host_counts = (
    df[df['Host_order'] != 'Unknown']
      .groupby(['Host_order','system_type'])
      .size()
      .unstack(fill_value=0)
)
# Sort host orders by total desc
host_counts['total'] = host_counts.sum(axis=1)
host_counts = host_counts.sort_values('total', ascending=False).drop(columns='total')

# 2) Create a 1×2 subplot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("a) Disease‐causing Agents by System",
                    "b) Host Plant Orders by System"),
    column_widths=[0.3, 0.7]
)

# --- Left panel: agents ---
for sys, color in zip(["Natural","Ag"], [WILD_COLOR, AG_COLOR]):
    fig.add_trace(
        go.Bar(
            x=agent_counts.index,
            y=agent_counts[sys],
            name=sys,
            marker_color=color
        ),
        row=1, col=1
    )

fig.update_xaxes(title_text="Disease‐causing Agent", row=1, col=1)
fig.update_yaxes(title_text="Observations", row=1, col=1)

# --- Right panel: host orders ---
for sys, color, showleg in zip(
    ["Natural","Ag"], [WILD_COLOR, AG_COLOR], [True, True]
):
    fig.add_trace(
        go.Bar(
            x=host_counts.index,
            y=host_counts[sys],
            name=sys,
            marker_color=color,
            showlegend=showleg  # legend only once
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="Host plant order", row=1, col=2, tickangle=45)
fig.update_yaxes(title_text="Observations", row=1, col=2)

# 3) Final layout tweaks
fig.update_layout(
    barmode='stack',
    template='simple_white',
    height=600, width=1000,
    legend=dict(x=0.75, y=0.95)
)

# 4) Render inline
import plotly.io as pio
pio.renderers.default = "notebook_connected"
fig.show()